In [33]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from skimage import io
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


In [25]:

# define the path to the input images
input_path = r"C:\Users\user\Documents\GitHub\Hand Gesture\Hand-Gesture\input"

# define the path to the output preprocessed images
output_path = r"C:\Users\user\Documents\GitHub\Hand Gesture\Hand-Gesture\output"

# define the target size of the preprocessed images
target_size = (64, 64)

# define the preprocessing functions


def convert_to_gray(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.convertScaleAbs(gray)
    return gray


def apply_gaussian_blur(img):
    blurred = cv2.GaussianBlur(img, (5, 5), 0)
    return blurred


def apply_threshold(img):
    _, thresh = cv2.threshold(
        img, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    return thresh


def find_contours(img):
    contours, hierarchy = cv2.findContours(
        img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours, hierarchy


def find_largest_contour(contours):
    max_contour = max(contours, key=cv2.contourArea)
    return max_contour


def create_mask(img, contour):
    mask = cv2.drawContours(np.zeros_like(
        img), [contour], 0, (255, 255, 255), -1)
    return mask


def apply_mask(img, mask):
    masked_image = cv2.bitwise_and(img, img, mask=mask)
    return masked_image


def resize_image(img, size):
    resized = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
    return resized


def normalize_image(img):
    normalized = img.astype("float32") / 255.0
    return normalized


def get_hog_features(img):
    win_size = (64, 64)
    block_size = (16, 16)
    block_stride = (8, 8)
    cell_size = (8, 8)
    nbins = 9
    hog = cv2.HOGDescriptor(win_size, block_size,
                            block_stride, cell_size, nbins)
    hog_feats = hog.compute(img)
    return hog_feats


def get_sift_features(img):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)
    return descriptors

def split_dataset(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

def svm_model(X_train, y_train, X_test):
    clf = svm.SVC(kernel='linear')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred
    
def calc_accuracy(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [26]:
#preprocessing phase
load_features_from_file=False
save_feature=False
all_feats = []
all_imgs = []


# set the path to the dataset folder
dataset_folder = "./input"
# loop through the images in the dataset folder
# for filename in os.listdir(dataset_folder):
#     if filename.endswith(".jpg"):  # check if the file is an image
#         filepath = os.path.join(dataset_folder, filename)
#         image = io.imread(filepath, as_gray=True)  # read the image in grayscale
#         blurred = apply_gaussian_blur(gray)
#         thresh = apply_threshold(blurred)
#         contours, hierarchy = find_contours(thresh)
#         max_contour = find_largest_contour(contours)
#         mask = create_mask(gray, max_contour)
#         masked_image = apply_mask(img, mask)
#         resized = resize_image(masked_image, target_size)
#         normalized = normalize_image(resized)
#         all_imgs.append(normalized)


In [27]:



# set the parameters for HoG
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (3, 3)
visualize = False
transform_sqrt = False
normalize = True



if(load_features_from_file != True):
    # loop through the images in the dataset folder
    for filename in os.listdir(dataset_folder):
        print(filename)
        if filename.endswith(".jpg") or filename.endswith(".JPG"):  # check if the file is an image
            filepath = os.path.join(dataset_folder, filename)
            image = io.imread(filepath, as_gray=True)  # read the image in grayscale
            label = np.array([int(filename[0])])
           
            
            #calculate the HoG features for the image
            hog_result = hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell,
                              cells_per_block=cells_per_block, visualize=visualize,
                              transform_sqrt=transform_sqrt, block_norm='L2-Hys', feature_vector=True)

            # unpack the HoG result into features and hog_image variables if necessary
            if visualize:
                features, hog_image = hog_result
            else:
                features = hog_result
            # do something with the HoG features, such as save to a file or use in a machine learning model
            print(filename, features)
            print(all(features)==0.0 or all(features)==0.)
            feats = np.concatenate((label, features.ravel()))
            
            print(feats)
            all_feats.append(feats)
           
else:
    features_array = np.loadtxt("hog_features.txt")

features_array = np.array(all_feats)
# # apply PCA to the HoG features
# pca = PCA(n_components=50)  # set the number of components to keep
# features_pca = pca.fit_transform(features_array)
if(save_feature):
    # convert the features list to a numpy array
    
    # save the features to a file
    np.savetxt("hog_features.txt", features_array)


print("Feature Extraction Finished")

# convert the list of features to a numpy array
#all_feats = np.array(all_feats)




0_men (1).JPG
0_men (1).JPG [0. 0. 0. ... 0. 0. 0.]
True
[0. 0. 0. ... 0. 0. 0.]
0_men (2).JPG
0_men (2).JPG [0. 0. 0. ... 0. 0. 0.]
True
[0. 0. 0. ... 0. 0. 0.]
0_men (3).JPG
0_men (3).JPG [0.         0.         0.         ... 0.10049795 0.0355337  0.02025369]
True
[0.         0.         0.         ... 0.10049795 0.0355337  0.02025369]
0_men (4).JPG
0_men (4).JPG [0.         0.         0.         ... 0.09942483 0.05240149 0.        ]
True
[0.         0.         0.         ... 0.09942483 0.05240149 0.        ]
0_men (5).JPG
0_men (5).JPG [0. 0. 0. ... 0. 0. 0.]
True
[0. 0. 0. ... 0. 0. 0.]
1_men (1).JPG
1_men (1).JPG [0.21830267 0.         0.18176215 ... 0.         0.         0.        ]
True
[1.         0.21830267 0.         ... 0.         0.         0.        ]
1_men (2).JPG
1_men (2).JPG [0.         0.         0.         ... 0.06509978 0.0343106  0.        ]
True
[1.         0.         0.         ... 0.06509978 0.0343106  0.        ]
1_men (3).JPG
1_men (3).JPG [0.         0.       

In [ ]:

# Extract the first column as Y (labels)
Y = features_array[:, 0]
# Extract the other columns as X (features)
X = features_array[:, 1:]

# splitting data into train and test
X_train, X_test, y_train, y_test = split_dataset(X, Y)

pca = PCA().fit(X_train)

# plot the cumulative explained variance ratio
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')
plt.show()
# apply SVM
y_pred = svm_model(X_train, y_train, X_test)



In [31]:
# calc accuracy
accuracy = calc_accuracy(y_test, y_pred)
print(accuracy)

0.4
